In [ ]:
# test 모델 만들어보기 (gpt)

In [ ]:
# NIH 데이터셋을 기반으로 test데이터 실험해보기

import os
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import torch.nn as nn

# 데이터셋 경로 설정
image_dir = 'path_to_chest_xray_images'
csv_file = 'path_to_labels.csv'

# CSV 파일 읽기
df = pd.read_csv(csv_file)

# 흉부 X-ray 데이터셋 클래스 정의
class ChestXrayDataset(Dataset):
    def __init__(self, csv_file, image_dir, transform=None):
        self.labels_df = pd.read_csv(csv_file)
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.labels_df.iloc[idx, 0])
        image = Image.open(img_name).convert('RGB')
        label = self.labels_df.iloc[idx, 1:].values
        label = label.astype('float')

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label)

# 데이터 변환 설정
data_transforms = {
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# 테스트 데이터셋 및 데이터 로더 설정
test_dataset = ChestXrayDataset(csv_file, image_dir, transform=data_transforms['test'])
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# 모델 설정 (Pretrained ResNet 사용)
model = models.resnet18(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(df.columns) - 1)  # 라벨의 수에 맞게 출력 차원 설정

model = model.to('cuda')

# 모델 가중치 불러오기
model.load_state_dict(torch.load('chest_xray_model.pth'))
model.eval()  # 모델을 평가 모드로 설정

In [ ]:
# 모델 테스트 후 바뀐 경로의 이미지를 불러왔을 때 결과예측
import torch
from torchvision import transforms
from PIL import Image

# GPU 사용 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 모델 설정 (ResNet)
model = models.resnet18(weights='DEFAULT')
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(df.columns) - 1)  # 추후 classificaion 원하는 숫자에 맞춰 변경

model = model.to(device)

# 저장된 모델 가중치 로드
model.load_state_dict(torch.load('chest_xray_model.pth'))
model.eval()  # 모델을 평가 모드로 설정

# 예측 함수 정의
def predict(image_path, model, transform):
    # 이미지 로드 및 전처리
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)  # 배치 차원 추가 # 배치의 차원을 추가한다는게 흑백이면 2차원 rgb면 3차원이라는 소리인건지...
    image = image.to(device)

    # 모델 예측
    with torch.no_grad():
        outputs = model(image)
        preds = (outputs > 0.5).float()

    return preds.cpu().numpy()

# 데이터 변환 설정 (평가용) # 사이즈는 이미 변경되어 있는 이미지라 필요성을 못느끼겠지만 Normalize의 필요성??...
data_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 예측할 이미지 경로 설정
image_path = 'path_to_new_image.jpg' # 경로는 저장된 경로가 아닌 사진 업로드시 저장될 경로로 변경

# 예측 수행
predictions = predict(image_path, model, data_transform)
print("Predictions:", predictions)